In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
# Load training and test data:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

WARNING - (py.warnings._showwarnmsg) - /var/folders/sk/1lq3mdt55q3579qwycd7hv4c0000gn/T/ipykernel_65391/1970558186.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')



In [86]:
# Select one station
station_name = 'Streeter Dr & Grand Ave'
# Ensure all dates are in datetime format
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

# Filter training and test data for the selected station
station_train_data = train[train.start_station_name == station_name].drop(columns=['start_station_name'])
station_test_data = test[test.start_station_name == station_name].drop(columns=['start_station_name'])
# Rename columns to match NeuralProphet requirements
station_train_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)
station_test_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)
# Filter only the required columns
station_train_data = station_train_data[['ds', 'y', 'temp_min_c']]
station_test_data = station_test_data[['ds', 'y', 'temp_min_c']]

# Debugging: Print first few rows of the station-specific data
print(f"Train Data for {station_name}:\n", station_train_data.head())
print(f"Test Data for {station_name}:\n", station_test_data.head())

# Ensure 'y' column is present
assert 'y' in station_train_data.columns, "Error: 'y' column missing in train data!"
assert 'y' in station_test_data.columns, "Error: 'y' column missing in test data!"

# Initialize the NeuralProphet model
model = NeuralProphet(epochs=100)

# Add 'temp_min_c' as a future regressor
model.add_future_regressor('temp_min_c')

# Train the model
metrics = model.fit(station_train_data, freq='D')

# Store model stats
model_stats = metrics

# Use test data as future dataframe
future = station_test_data[['ds', 'temp_min_c']].copy()
future['y'] = station_test_data['y']

# Predict future values
forecast = model.predict(future)

# Merge forecast with actual values
result = forecast[['ds', 'yhat1']].merge(station_test_data[['ds', 'y']], on='ds', how='left')


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.958% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._i

Train Data for Streeter Dr & Grand Ave:
              ds   y  temp_min_c
178  2020-01-01  39        -5.5
625  2020-01-02  45         2.0
1086 2020-01-03  31         0.0
1493 2020-01-04  15        -3.5
2012 2020-01-05  30        -4.1
Test Data for Streeter Dr & Grand Ave:
              ds   y  temp_min_c
242  2024-01-01  16        -1.9
818  2024-01-02  33        -2.8
1422 2024-01-03  20        -0.6
2038 2024-01-04  19        -1.2
2650 2024-01-05  30        -1.8


Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.




Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.892% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/

Predicting: 45it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [ ]:
from sklearn.metrics import r2_score

# Extract actual and predicted values
y_actual = result['y'].dropna()  # Drop NaNs to ensure alignment
y_predicted = result.loc[y_actual.index, 'yhat1']  # Ensure same index alignment

# Compute R²
r2 = r2_score(y_actual, y_predicted)

print("R² Score:", r2)

,MAE,RMSE,Loss,RegLoss,epoch
0,1822.194946,1939.123413,2.188069,0.0,0
1,1249.833984,1399.503052,1.446091,0.0,1
2,700.542053,847.964417,0.743133,0.0,2
3,280.941071,381.810638,0.239470,0.0,3
4,153.527588,194.777817,0.099685,0.0,4
...,...,...,...,...,...
95,79.858459,114.581047,0.039329,0.0,95
96,79.972687,114.552452,0.039397,0.0,96
97,79.794434,113.430634,0.039362,0.0,97
98,79.693367,115.312698,0.039373,0.0,98


In [9]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.metrics import r2_score

# Load training and test data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Ensure all dates are in datetime format
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

# Define the specific station to analyze
station_name = "Clark St & Elm St"

# Filter training and test data for the selected station
station_train_data = train[train.start_station_name == station_name].drop(columns=['start_station_name'])
station_test_data = test[test.start_station_name == station_name].drop(columns=['start_station_name'])

# Rename columns to match NeuralProphet requirements
station_train_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)
station_test_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)

# Filter only the required columns
station_train_data = station_train_data[['ds', 'y', 'temp_min_c']]
station_test_data = station_test_data[['ds', 'y', 'temp_min_c']]

# Ensure no negative or zero values in y (total rides)
station_train_data = station_train_data[station_train_data['y'] > 0]
station_test_data = station_test_data[station_test_data['y'] > 0]

# Initialize the NeuralProphet model with normalization and robust loss
model = NeuralProphet(
    epochs=100,
    yearly_seasonality='auto',
    weekly_seasonality='auto',
    loss_func="Huber",  # More robust loss to avoid extreme errors
    normalize='soft'  # Normalize data for better training
)

# Add 'temp_min_c' as a future regressor
model.add_future_regressor('temp_min_c')

# Train the model with early stopping
model.fit(station_train_data, freq='D', early_stopping=True)

# Prepare future dataframe
future = station_test_data[['ds', 'temp_min_c']].copy()
future['y'] = station_test_data['y']

# Predict future values
forecast = model.predict(future)

# Ensure predictions are non-negative
forecast['yhat1'] = forecast['yhat1'].clip(lower=0)

# Merge forecast with actual values
result = forecast[['ds', 'yhat1']].merge(station_test_data[['ds', 'y']], on='ds', how='left')
result['station_name'] = station_name

# Compute R² score
y_actual = result['y'].dropna()
y_predicted = result.loc[y_actual.index, 'yhat1']

r2 = r2_score(y_actual, y_predicted)

# Convert results into DataFrame for display
r2_df = pd.DataFrame([[station_name, r2]], columns=['Station Name', 'R² Score'])

WARNING - (py.warnings._showwarnmsg) - /var/folders/sk/1lq3mdt55q3579qwycd7hv4c0000gn/T/ipykernel_440/477343962.py:7: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.863% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/l

Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.




Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.452% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/

Predicting: 46it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
